### Here our Results

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import sys
import os
import requests
import swifter

from tqdm import tqdm
from pygments.lexers.webassembly import keywords

from src.data.TMDB_Movies import get_data
from src.data.TMDB_Movies import get_collection

current_dir = os.path.abspath("")
sys.path.append(current_dir)
data_dir = os.path.join(current_dir, "data")


In [2]:
#we want to organize the data in tvclusters into a dataframe and sort by actor


# Read the text file into a DataFrame
tv_tropes_path = os.path.join(data_dir, "MovieSummaries/tvtropes.clusters.txt")
tv_clusters = pd.read_csv(tv_tropes_path, sep='\t', header=None, names=['trope', 'details'])

# Parse the JSON string in the 'details' column and expand it into separate columns
tv_clusters = tv_clusters.join(tv_clusters['details'].apply(json.loads).apply(pd.Series))

# Drop the 'details' column as it's no longer needed
tv_clusters.drop(columns=['details'], inplace=True)

# Display the DataFrame


tv_clusters_char = tv_clusters.sort_values(by='char')

tv_clusters_char = tv_clusters_char.reset_index(drop=True)

len(tv_clusters_char)


501

In [ ]:
# try to get the movies with sequels 

char_count = tv_clusters_char.groupby('char').size()

multiple_char = char_count[char_count > 1]

multiple_char


In [70]:
tv_clusters_char = tv_clusters_char[tv_clusters_char['char'].isin(multiple_char.index)]

tv_clusters_char = tv_clusters_char.reset_index(drop=True)
tv_clusters_char.head(25)


,trope,char,movie,id,actor
0,arrogant_kungfu_guy,Anakin Skywalker,Star Wars Episode III: Revenge of the Sith,/m/02sg5ry,Hayden Christensen
1,child_prodigy,Anakin Skywalker,Star Wars Episode I: The Phantom Menace,/m/02sg5rn,Jake Lloyd
2,byronic_hero,Anakin Skywalker,Star Wars Episode III: Revenge of the Sith,/m/02sg5ry,Hayden Christensen
3,gadgeteer_genius,Anakin Skywalker,Star Wars Episode I: The Phantom Menace,/m/02sg5rn,Jake Lloyd
4,warrior_poet,Arthur Burns,The Proposition,/m/0k66rt,Danny Huston
5,cultured_badass,Arthur Burns,The Proposition,/m/0k66rt,Danny Huston
6,jerk_jock,Biff Tannen,Back to the Future,/m/0jty60,Thomas F. Wilson
7,bully,Biff Tannen,Back to the Future,/m/0jty60,Thomas F. Wilson
8,crazy_jealous_guy,Bill,Kill Bill Volume 1,/m/0j_fvv,David Carradine
9,heartbroken_badass,Bill,Kill Bill Volume 1,/m/0j_fvv,David Carradine


In [82]:
#get only movies with sequels 
sequel_char = tv_clusters_char.groupby('char')['movie'].nunique()

sequel_char = sequel_char[sequel_char > 1]  

tv_clusters_sequel = tv_clusters_char[tv_clusters_char['char'].isin(sequel_char.index)]


tv_clusters_sequel = tv_clusters_sequel.reset_index(drop=True)


In [88]:
tv_clusters_sequel.head(35)

,trope,char,movie,id,actor
0,arrogant_kungfu_guy,Anakin Skywalker,Star Wars Episode III: Revenge of the Sith,/m/02sg5ry,Hayden Christensen
1,child_prodigy,Anakin Skywalker,Star Wars Episode I: The Phantom Menace,/m/02sg5rn,Jake Lloyd
2,byronic_hero,Anakin Skywalker,Star Wars Episode III: Revenge of the Sith,/m/02sg5ry,Hayden Christensen
3,gadgeteer_genius,Anakin Skywalker,Star Wars Episode I: The Phantom Menace,/m/02sg5rn,Jake Lloyd
4,dumb_blonde,Cindy Campbell,Scary Movie 3,/m/0k2z05,Anna Faris
5,dumb_blonde,Cindy Campbell,Scary Movie 4,/m/0k2y_7,Anna Faris
6,corrupt_corporate_executive,Cutler Beckett,Pirates of the Caribbean: Dead Man's Chest,/m/0k1xyw,Tom Hollander
7,corrupt_corporate_executive,Cutler Beckett,Pirates of the Caribbean: At World's End,/m/0k1xwx,Tom Hollander
8,charmer,Han Solo,Star Wars Episode IV: A New Hope,/m/0k3r1_,Harrison Ford
9,loveable_rogue,Han Solo,Star Wars Episode IV: A New Hope,/m/0k3r1_,Harrison Ford


### 2.1 Data Collection

Here we download the data from TMDB.


In [5]:
keywords_name = ["sequel", "book", "comic", "remake"]
keywords = [9663, 818, 9717, 9714] #keywords for the movies corresponding to the sequels, book or novel adaptations, and based on comics, and remakes

for i, keyword in enumerate(keywords) :
    get_data(keywords_name[i], "2013-01-01", keyword)

100%|██████████| 43/43 [00:07<00:00,  5.70it/s]


JSONDecodeError: Extra data: line 1 column 86 (char 85)

### 2.2 Use TMDB collection to get group the sequels together.

In [ ]:
sequels_movies = pd.read_csv('data/movie_with_keyword_sequel.csv')
get_collection(sequels_movies)


### 2.3 Find the movies in the Dataset using TMDB collection

In [85]:
url = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&format=json&titles=Harry%20Potter%20and%20the%20Deathly%20Hallows:%20Part%201"
requests.get(url).json()

{'batchcomplete': '',
 'query': {'pages': {'17445028': {'pageid': 17445028,
    'ns': 0,
    'title': 'Harry Potter and the Deathly Hallows: Part 1'}}}}

In [26]:
def get_wikipedia_id_from_title(title, year):
    api_key = 
    title += " film, " + str(year)
    title.replace(" ", "%20")
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&titles={title}"
    
    
    language_code = 'en'
    base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
    endpoint = '/search/page'
    url = base_url + language_code + endpoint
    parameters = {'q': title, 'limit': 1}
    page = requests.get(url, params=parameters).json().get('pages')
    id = page[0].get('id', 0) if page is not None else 0 
    id = int(id) if id else None
    print(id)
    return id

In [27]:
get_wikipedia_id_from_title("Superman IV: The Quest for Peace film")

In [53]:
import time
from more_itertools import sliced

sequel_collections = pd.read_csv('data/collections/sequels.csv')

tqdm.pandas()
sequel_collections = sequel_collections.drop("Wikipedia movie ID", axis=1)

slices = sliced(seq=range(len(sequel_collections)), n=50)
sequel_collections_with_wiki_id = pd.DataFrame()

i=0
for index in tqdm(slices, total=40):
    time.sleep(10)
    if i < 8 or i > 9:
        i+=1
        continue
    chunk = sequel_collections.iloc[index].copy()
    chunk["Wikipedia movie ID"] = chunk["title"].apply(get_wikipedia_id_from_title) # Apply function/transformation here
    sequel_collections_with_wiki_id = pd.concat([sequel_collections_with_wiki_id, chunk], axis=0, ignore_index=True, sort=False)
    chunk.to_csv(f'df{i}.csv')
    i+=1

100%|██████████| 40/40 [01:36<00:00,  2.41s/it]


In [55]:
sequel_collections_with_wiki_id = pd.DataFrame()
for i in range(40):
    df = pd.read_csv(f'df{i}.csv')
    sequel_collections_with_wiki_id = pd.concat([sequel_collections_with_wiki_id, df], axis=0, ignore_index=True, sort=False)

sequel_collections_with_wiki_id.to_csv('data/collections/sequels_with_wiki_id.csv')

In [58]:
sequel_collections_with_wiki_id["Wikipedia movie ID"] = pd.to_numeric(sequel_collections_with_wiki_id["Wikipedia movie ID"], errors='coerce')
sequel_collections_with_wiki_id = sequel_collections_with_wiki_id.dropna(subset=['Wikipedia movie ID'])
sequel_collections_with_wiki_id['Wikipedia movie ID'] = sequel_collections_with_wiki_id['Wikipedia movie ID'].astype(int)

/tmp/ipykernel_10618/1352926878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequel_collections_with_wiki_id['Wikipedia movie ID'] = sequel_collections_with_wiki_id['Wikipedia movie ID'].astype(int)


In [ ]:
sequel_collections_with_wiki_id

In [66]:
sequel_collections_with_wiki_id

,Unnamed: 0,id,release_date,original_title,title,collection,collection_id,Wikipedia movie ID
0,0,8835,2001-07-13,Legally Blonde,Legally Blonde,Legally Blonde Collection,86024,444267
1,1,10327,2003-07-02,"Legally Blonde 2: Red, White & Blonde","Legally Blonde 2: Red, White & Blonde",Legally Blonde Collection,86024,447672
2,2,528320,NaN,Legally Blonde 3,Legally Blonde 3,Legally Blonde Collection,86024,444267
3,3,11,1977-05-25,Star Wars,Star Wars,Star Wars Collection,10,52549
4,4,1891,1980-05-20,The Empire Strikes Back,The Empire Strikes Back,Star Wars Collection,10,53964
...,...,...,...,...,...,...,...,...
495,495,32115,1961-04-04,Carry On Regardless,Carry On Regardless,Carry On Collection,918027,3694594
496,496,26190,1962-03-12,Carry On Cruising,Carry On Cruising,Carry On Collection,918027,3694699
497,497,32105,1964-02-23,Carry On Jack,Carry On Jack,Carry On Collection,918027,36999
498,498,5061,1966-12-10,Carry On Don't Lose Your Head,Carry On Don't Lose Your Head,Carry On Collection,918027,3737477


In [71]:
movie_df = pd.read_csv('data/MovieSummaries/movie.metadata.tsv', sep='\t', header=None)

movie_df.rename(columns={0: 'Wikipedia movie ID', 1: "Freebase movie ID", 2: "Movie name",  3: "Movie release date", 4: "Movie box office revenue", 5: "Movie runtime", 6: "Movie languages", 7: "Movie countries", 8: "Movie genres"}, inplace=True)

collection_df = sequel_collections_with_wiki_id.drop(["title", "Unnamed: 0", "original_title", "release_date"], axis=1)

movie_df_sequel = movie_df.join(collection_df.set_index('Wikipedia movie ID'), on="Wikipedia movie ID", how='inner')

In [72]:
movie_df_sequel

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,id,collection,collection_id
179,33872851,/m/09vnx57,The Lone Wolf Meets a Lady,1940-05-30,NaN,71.0,"{""/m/0349s"": ""Greek Language"", ""/m/02h40lc"": ""...","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/028v3"": ""Det...",85231,Lone Wolf Collection,395433
309,6273971,/m/0f_17k,"The Prophet, the Gold and the Transylvanians",1978,NaN,NaN,"{""/m/02hxc3j"": ""Romanian Language""}","{""/m/06c1y"": ""Romania""}","{""/m/01z4y"": ""Comedy""}",147683,Transylvanians Trilogy,620931
771,399402,/m/023p33,Cinderella,1950-02-15,8.500000e+07,74.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02l7c8"": ""Romance Film"", ""/m/01hmnh"": ""Fa...",11224,Cinderella Collection,55419
902,3694061,/m/09vttt,Carry On Teacher,1959-08,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/0gf28"": ""Parody"", ""/m/01z4y"": ""Comedy""}",32113,Carry On Collection,918027
1045,3504034,/m/09h37b,The Mummy's Ghost,1944-07-07,NaN,61.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03jp5h0"": ""Archaeology"", ""/m/03npn"": ""Hor...",29243,The Mummy (Universal) Collection,221544
...,...,...,...,...,...,...,...,...,...,...,...,...
80920,174251,/m/017jd9,The Lord of the Rings: The Return of the King,2003-12-17,1.119930e+09,250.0,"{""/m/05p2d"": ""Old English language"", ""/m/02h40...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/0hj3n2k"": ""Fantasy Adventure"", ""/m/03k9fj...",122,The Lord of the Rings Collection,119
81048,1853749,/m/0615h6,Frankenstein Conquers the World,1965-08-08,NaN,94.0,"{""/m/03_9r"": ""Japanese Language"", ""/m/02h40lc""...","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",3160,Frankenstein (Toho) Collection,491768
81225,2303174,/m/0729rh,Ice Age: The Meltdown,2006-03-01,6.553882e+08,90.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03k9fj"": ""Adventure"", ""/m/0hj3myq"": ""Chil...",950,Ice Age Collection,8354
81621,2472440,/m/07gf00,Turbulence,1997-01-10,1.153824e+07,101.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/01drsx"": ""Disast...",34314,Turbulence Collection,166379


In [73]:
movie_df_sequel.to_csv('data/movie_sequel_tmdb.csv')